In [2]:
import pandas as pd
import numpy as np
import pickle
import re
import timeit
import spacy

import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.utils import simple_preprocess
from gensim.models import LdaMulticore, ldaseqmodel
from nltk.corpus import stopwords
from gensim.matutils import hellinger
import helper as he
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
# # loading our corpus and dictionary
# try:
#     dictionary = Dictionary.load('datasets/news_dictionary')
# except FileNotFoundError as e:
#     raise ValueError("SKIP: Please download the Corpus/news_dictionary dataset.")
# corpus = bleicorpus.BleiCorpus('datasets/news_corpus')
# # it's very important that your corpus is saved in order of your time-slices!

# time_slice = [438, 430, 456]

In [4]:
with open('../data/preprocessed_data/aadhar_corp.pkl','rb') as f:
    data_lemmatized, id2word, corpus = pickle.load(f)
    
with open('../data/massmedia-data/aadhar_date_indexed.pkl','rb') as f:
    _, time_index_arr = pickle.load(f)

In [5]:
def generate_timeslice_data(index_arr):
    arr = []
    for i in index_arr.values():
        arr.append(i)
        
    diff_arr=[]
    for i in range(1,len(arr)):
        diff_arr.append(arr[i]-arr[i-1])
    return diff_arr

In [6]:
time_slices = generate_timeslice_data(time_index_arr)

In [7]:
lda = LdaMulticore(corpus, num_topics=35, id2word=id2word,
                   workers=3, chunksize=2000, passes=10, batch=False)

2019-12-04 19:39:35,215 : INFO : using symmetric alpha at 0.02857142857142857
2019-12-04 19:39:35,216 : INFO : using symmetric eta at 0.02857142857142857
2019-12-04 19:39:35,217 : INFO : using serial LDA version on this node
2019-12-04 19:39:35,232 : INFO : running online LDA training, 35 topics, 10 passes over the supplied corpus of 13908 documents, updating every 6000 documents, evaluating every ~13908 documents, iterating 50x with a convergence threshold of 0.001000
2019-12-04 19:39:35,247 : INFO : training LDA model using 3 processes
2019-12-04 19:39:35,291 : INFO : PROGRESS: pass 0, dispatched chunk #0 = documents up to #2000/13908, outstanding queue size 1
2019-12-04 19:39:35,385 : INFO : PROGRESS: pass 0, dispatched chunk #1 = documents up to #4000/13908, outstanding queue size 2
2019-12-04 19:39:35,482 : INFO : PROGRESS: pass 0, dispatched chunk #2 = documents up to #6000/13908, outstanding queue size 3
2019-12-04 19:39:35,506 : INFO : PROGRESS: pass 0, dispatched chunk #3 = do

In [ ]:
# ldaseq = ldaseqmodel.LdaSeqModel(corpus=corpus, id2word=id2word, time_slice=time_slices, num_topics=35)
ldaseq = ldaseqmodel.LdaSeqModel(lda_model=lda, corpus=corpus, time_slice=time_slices)

2019-12-04 19:43:35,978 : WARNING : no word id mapping provided; initializing from corpus, assuming identity
2019-12-04 19:43:36,100 : INFO : using symmetric eta at 0.1
2019-12-04 19:43:36,102 : INFO : using serial LDA version on this node
2019-12-04 19:43:36,108 : INFO : running online (multi-pass) LDA training, 10 topics, 10 passes over the supplied corpus of 13908 documents, updating model once every 2000 documents, evaluating perplexity every 13908 documents, iterating 50x with a convergence threshold of 0.001000
2019-12-04 19:43:36,117 : INFO : PROGRESS: pass 0, at document #2000/13908
2019-12-04 19:43:38,031 : INFO : merging changes from 2000 documents into a model of 13908 documents
2019-12-04 19:43:38,034 : INFO : topic #1 (0.010): 0.031*"35" + 0.026*"0" + 0.017*"7" + 0.012*"16" + 0.011*"113" + 0.010*"395" + 0.009*"146" + 0.009*"392" + 0.009*"36" + 0.008*"118"
2019-12-04 19:43:38,036 : INFO : topic #2 (0.010): 0.019*"35" + 0.019*"0" + 0.017*"93" + 0.014*"36" + 0.013*"348" + 0.0